Perhaps add data streaming?

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time as timer
import pickle

Input_File = 'ChangeDataset622v2.pickle'

def ReadPickle(filename: str) -> dict:
  '''Reads in data from given pickle files, outputs a dictionary'''
  try:
    Data = pd.read_pickle(filename)
  except FileNotFoundError:
    raise FileNotFoundError(f'Error reading {filename}')
  return Data

In [4]:
rmin = 1e-6
rmax = 1.1
nr = 50
r, dr = np.linspace(rmin, rmax, nr, retstep = True)

#Time Grid
tmin = 2.9
tmax = 4.0
nt = 1000
time, dt = np.linspace(tmin, tmax, nt, retstep = True)

In [7]:
print(f"Loading Dataset -- '{Input_File}'")
t_Load1a = timer.time()
Data = ReadPickle(Input_File)
t_Load2a = timer.time()
print(f"Dataset took {round(t_Load2a-t_Load1a,2)}s to load")

D_data = np.array([np.tile(sample['Diffusion'],(1000,1)) for sample in Data])
V_data = np.array([np.tile(sample['Convection'],(1000,1)) for sample in Data])
#R_data = np.array([np.tile(sample['Rho'],(1000,1)) for sample in Data])
#T_data = np.array([np.tile(sample['Time'],(50, 1)).T for sample in Data])
N_data = np.array([Sample['Density'] for Sample in Data])
S_data = np.array([Sample['Source'].T for Sample in Data])

Database = np.array([np.array([D_data[i],V_data[i],N_data[i],S_data[i]]) for i in range(len(Data))])
print(Database.shape)
del Data
del D_data, V_data, N_data, S_data, #R_data, T_data

Loading Dataset -- 'ChangeDataset622v2.pickle'
Dataset took 2.56s to load
(10000, 4, 1000, 50)


In [ ]:
# data3 = ReadPickle('ChangeDatasetPre622v3b.pickle')

In [ ]:
# print(data3[0].keys())
# print(data3[0]['D'])


# data4 = []
# for i in range(len(data3)):
#     dataa = {}
#     dataa['Diffusion']= data3[i]['D']
#     dataa['Convection'] = data3[i]['V']
#     dataa['Rho'] = r
#     dataa['Time'] = time
#     dataa['Density'] = data3[i]['N']
#     dataa['Source'] = np.outer(data3[i]['ST'], data3[i]['SR'])
#     data4.append(dataa)
# data4 = np.array(data4)

In [8]:
def WritePickle(data_out: object, filename: str) -> None:
  '''Writes data to pickle file'''
  if not filename.endswith('.pickle'):
    filename += '.pickle'
  with open(filename, 'wb+') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data_out, f, pickle.HIGHEST_PROTOCOL)

# WritePickle(data4, 'ChangeDataset622v3.pickle')

In [ ]:
#This i will code myself later, had gpt code up a basic scaler to use for testing now

#############################GPT CODE FOR SCALING##############################

import numpy as np

# Assume Database is loaded with shape (num_samples, 4, 1000, 50)
print("Original shape:", Database.shape)

# Prepare an empty array to hold the standardized data
Database_channel_standardized = np.empty_like(Database)

# Loop over each channel to standardize it independently.
# Here axis=(0, 2, 3) computes statistics across all samples and the 2D dimensions for the given channel.
for c in range(Database.shape[1]):
    channel_data = Database[:, c, :, :]
    channel_mean = np.mean(channel_data)
    channel_std = np.std(channel_data)
    print(f"Channel {c}: mean = {channel_mean:.4f}, std = {channel_std:.4f}")
    
    Database_channel_standardized[:, c, :, :] = (channel_data - channel_mean) / channel_std
# Optionally, verify the results for one channel:
print("Channel 0 after standardization: mean = {:.4f}, std = {:.4f}".format(
    np.mean(Database_channel_standardized[:, 3, :, :]),
    np.std(Database_channel_standardized[:, 3, :, :])
))
#############################GPT CODE FOR SCALING##############################
del Database
del channel_data

Original shape: (10000, 4, 1000, 50)
Channel 0: mean = 0.9100, std = 0.5149
Channel 1: mean = 0.1383, std = 0.9820
Channel 2: mean = 37906458524158672896.0000, std = 26834145827980509184.0000
Channel 3: mean = 7390494301068411904.0000, std = 16673122821864118272.0000
Channel 0 after standardization: mean = 0.0000, std = 1.0000


In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Convolutional Variational Autoencoder:
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dim=640):
        super(VariationalAutoencoder, self).__init__()
        
        # Encoder:
        self.encoder_conv = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2, padding=1),# -> (8, 500, 25)
            nn.SELU(),
            nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),# -> (16, 250, 13)
            nn.SELU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),# -> (32, 125, 7)
            nn.SELU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),# -> (64, 63, 4)
            nn.SELU()
        )
        
        #Flattened conv feature size is 64*63*4 = 16,128
        self.fc1 = nn.Linear(64*63*4,128)
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_logvar = nn.Linear(128, latent_dim)
        
        #Decoder:
        self.decoder_fc = nn.Linear(latent_dim, 64*63*4)
        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=0),# -> (32, 125, 7)
            nn.SELU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=(1, 0)),# -> (16, 250, 13)
            nn.SELU(),
            nn.ConvTranspose2d(16, 8, kernel_size=3, stride=2, padding=1, output_padding=(1, 0)),# -> (8, 500, 25)
            nn.SELU(),
            nn.ConvTranspose2d(8, 4, kernel_size=3, stride=2, padding=1, output_padding=(1, 1))# -> (4, 1000, 50)
        )
    
    def encode(self, x):
        x = self.encoder_conv(x)
        x = x.view(x.size(0),-1)
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        x = self.decoder_fc(z)
        x = x.view(-1,64,63,4)
        x = self.decoder_conv(x)
        return x
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        decoded = self.decode(z)
        return decoded, mu, logvar

#Updated Loss Function using Mean Squared Error (MSE)
def vae_loss(decoded, original, mu, logvar, r, dt=1.0, dr=1.0, lambda_pinn=1e-3):

    #Unpack the decoded output
    D_hat = decoded[:,0:1]   #Diffusion
    v_hat = decoded[:,1:2]   #Convection
    N_hat = decoded[:,2:3]   #Density
    S_hat = decoded[:,3:4]   #Source

    #Find datatype and device
    dtype, device = N_hat.dtype, N_hat.device
    
    # tandard VAE loss
    N_true = original[:,2:3]
    MSE = F.mse_loss(N_hat, N_true, reduction='mean')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    #Collapse 2d transport arrays
    D1 = D_hat[:,:,0,:].unsqueeze(2)
    v1 = v_hat[:,:,0,:].unsqueeze(2)

    #Put rho array into proper form
    r = torch.tensor(r, dtype=torch.float32, device=device)
    r_b = r.view(1,1,1,-1).to(N_hat.device)

    #Finite Difference Kernels
    kernel_dt = torch.tensor(
        [[[[ -1/(2*dt) ], [ 0.], [ 1/(2*dt) ]]]],
        device=N_hat.device,
        dtype=dtype
    )  # shape (1,1,3,1)
    kernel_dr = torch.tensor(
        [[[[ -1/(2*dr), 0.,  1/(2*dr) ]]]],
        device=N_hat.device,
        dtype = dtype
    )  # shape (1,1,1,3)

    #dndt with Neumann BC
    x = F.pad(N_hat, (0,0, 1,1), mode='replicate')
    dndt = F.conv2d(x, kernel_dt)

    #Advective Term (1/r)*d_r(rvn)
    f_adv = r_b*v1*N_hat
    x = F.pad(f_adv, (1,1, 0,0), mode='replicate')  # pad radial dim
    dfr = F.conv2d(x, kernel_dr)
    adv = dfr/r_b

    #Diffusive Term: (1/r)*d_r(rD*d_r(n))
    x      = F.pad(N_hat, (1,1, 0,0), mode='replicate')
    dn_dr  = F.conv2d(x, kernel_dr)
    f_diff = r_b * D1 * dn_dr
    x      = F.pad(f_diff, (1,1, 0,0), mode='replicate')
    dfd    = F.conv2d(x, kernel_dr)
    diff   = dfd / r_b

    #Residual using Source term
    Resid = dndt + adv - diff - S_hat
    PINN = torch.mean(Resid**2)

    return MSE + KLD + lambda_pinn*PINN, PINN, MSE, KLD

#Custom Dataset to handle the numerical array
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)
        
    def __len__(self):
        return self.data.size(0)
    
    def __getitem__(self, idx):
        return self.data[idx], 0


In [44]:
#Training Setup
latent_dim = 640
batch_size = 32
lr_vae = 1e-3  #1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VariationalAutoencoder(latent_dim=640).to(device)
optimizer = optim.Adam(vae.parameters(), lr=lr_vae)

#Dataloaders
dataset = CustomDataset(Database_channel_standardized)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    total_samples = 0
    epoch_loss = 0
    epoch_loss_mse = 0
    epoch_loss_kld = 0
    epoch_loss_pinn = 0
    
    for batch_data, _ in dataloader:
        bs = batch_data.size(0)
        batch_data = batch_data.to(device)
        decoded_data, mu, logvar = vae(batch_data)
        
        loss, pinn, mse, kld = vae_loss(decoded_data, batch_data, mu, logvar, r, dt, dr, lambda_pinn=1e-3)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(vae.parameters(), max_norm=1.0)
        optimizer.step()
        
        epoch_loss += loss.item()*bs
        total_samples += bs
    
    avg_loss = epoch_loss / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}]: Average Loss: {avg_loss:.4f}" + ' '*45)
    print(f'Epoch [{epoch+1}/{num_epochs}]: MSE: {mse:.4f}, KLD: {kld:.4f}, PINN: {pinn:.4f}\n')
    
    prog_str = (f"Epoch [{epoch+1}/{num_epochs}]: "
                f"Avg: {avg_loss:.4f}, MSE: {mse:.4f}, KLD: {kld:.4f}, PINN: {pinn:.4f}")
    print(prog_str, end='\r', flush=True)
    
print("Training complete!" + ' '*70)

Epoch [1/100]: Average Loss: 153.7502                                             
Epoch [1/100]: MSE: 0.6875, KLD: 0.0003, PINN: 22.7467

Epoch [2/100]: Average Loss: 0.9378                                             
Epoch [2/100]: MSE: 1.2300, KLD: 0.0007, PINN: 43.1122

Epoch [3/100]: Average Loss: 2.7809                                             
Epoch [3/100]: MSE: 0.7421, KLD: 0.0004, PINN: 31.6220

Epoch [4/100]: Average Loss: 0.8636                                             
Epoch [4/100]: MSE: 0.7187, KLD: 0.0001, PINN: 30.7261

Epoch [5/100]: Average Loss: 0.8137                                             
Epoch [5/100]: MSE: 0.9512, KLD: 0.0001, PINN: 19.0208

Epoch [6/100]: Average Loss: 0.8303                                             
Epoch [6/100]: MSE: 0.5510, KLD: 0.0001, PINN: 16.9391

Epoch [7/100]: Average Loss: 0.7870                                             
Epoch [7/100]: MSE: 0.9015, KLD: 0.0001, PINN: 24.8232

Epoch [8/100]: Average Loss: 0.7862    

In [ ]:
torch.save(vae, 'VAE_Pretrained_PINN_v1.pt')

Add pinn residual scoring

In [ ]:
import torch
import matplotlib.pyplot as plt

#Set the VAE to evaluation mode
vae.eval()

#Specify the latent dimension (should match what you used during training)
latent_dim = 640

#Sample a single latent vector from a standard normal distribution and decode it.
with torch.no_grad():
    z1 = torch.randn(1, latent_dim, device=device)  #Sample one latent vector
    z2 = torch.randn(1, latent_dim, device=device)  #Sample one latent vector
    generated1 = vae.decode(z1)
    generated2 = vae.decode(z2)

#Remove the batch dimension and move the tensor to CPU
sample1 = generated1.squeeze(0).cpu()
sample2 = generated2.squeeze(0).cpu()
sample1_np = sample1.numpy()
sample2_np = sample2.numpy()

#Plotting
fig, axs = plt.subplots(2, 3, figsize=(10,7))
axs = axs.ravel()
id_num = np.random.randint(10000)


for i in range(6):
    im = axs[i].imshow(sample1_np[i], aspect='auto', cmap='viridis')
    axs[i].set_title(f'Generated Channel {i}')
    axs[i].axis('off')
    fig.colorbar(im, ax=axs[i], fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()

fig.savefig(f'GeneratedChannelsv{id_num}.png', dpi=300, bbox_inches='tight')
    
fig2, axs2 = plt.subplots(2, 3, figsize=(10,7))
axs2 = axs2.ravel()

for i in range(6):
    im = axs2[i].imshow(Database_channel_standardized[np.random.randint(1000)][i], aspect='auto', cmap='viridis')
    axs2[i].set_title(f'Base Channel {i}')
    axs2[i].axis('off')
    fig2.colorbar(im, ax=axs2[i], fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()

fig.savefig(f'BaseChannel{id_num}.png', dpi=300, bbox_inches='tight')

fig, axs = plt.subplots(2, 3, figsize=(10,7))
axs = axs.ravel()

for i in range(6):
    im = axs[i].imshow((sample1_np[i] - sample2_np[i]), aspect='auto', cmap='viridis')
    axs[i].set_title(f'Variance Channel {i}')
    axs[i].axis('off')
    fig.colorbar(im, ax=axs[i], fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()

fig.savefig(f'VarianceChannel{id_num}.png', dpi=300, bbox_inches='tight')


In [ ]:
#Poolng, fix noise, etc.